In [14]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

Automatic pdb calling has been turned OFF
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import sys
from pathlib import Path
import neptune # for logging progress and results
from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import set_environment_variables, neptune_output_figures
neptune_kwargs = {'project':"commander.pho/PhoDibaLongShort2023",
'api_token':"eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxOGIxODU2My1lZTNhLTQ2ZWMtOTkzNS02ZTRmNzM5YmNjNjIifQ=="}

# required to enable non-blocking interaction:
%gui qt5

from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd
from attrs import define, field, fields, Factory

# Pho's Formatting Preferences
# from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
# set_pho_preferences_concise()

## Pho's Custom Libraries:
from pyphocorehelpers.general_helpers import CodeConversion
from pyphocorehelpers.function_helpers import function_attributes
from pyphocorehelpers.print_helpers import print_keys_if_possible, print_value_overview_only, document_active_variables, objsize, print_object_memory_usage, debug_dump_object_member_shapes, TypePrintMode
from pyphocorehelpers.print_helpers import get_now_day_str, get_now_time_str, get_now_time_precise_str
from pyphocorehelpers.programming_helpers import IPythonHelpers
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager

# NeuroPy (Diba Lab Python Repo) Loading
# from neuropy import core
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.core.epoch import NamedTimerange, Epoch
from neuropy.core.ratemap import Ratemap
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass

## For computation parameters:
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.utils.dynamic_container import DynamicContainer
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core.neurons import NeuronType

from pyphocorehelpers.print_helpers import generate_html_string

# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot, create_daily_programmatic_display_function_testing_folder_if_needed, session_context_to_relative_path
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_load_session, batch_extended_computations, SessionBatchProgress, batch_programmatic_figures, batch_extended_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap
from pyphoplacecellanalysis.Pho2D.PyQtPlots.Extensions.pyqtgraph_helpers import build_pyqtgraph_epoch_indicator_regions # Add session indicators to pyqtgraph plot
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _helper_add_long_short_session_indicator_regions
import pyphoplacecellanalysis.External.pyqtgraph as pg
# 'time_bin_indices': valid_time_bin_indicies, 'posterior_to_pf_mean_surprise': one_left_out_posterior_to_pf_surprises_mean, 'posterior_to_scrambled_pf_mean_surprise': one_left_out_posterior_to_scrambled_pf_surprises_mean}
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_long_short_expected_vs_observed_firing_rates
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_long_short_surprise_difference_plot

from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import _update_pipeline_missing_preprocessing_parameters
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons

from scripts.run_BatchAnalysis import post_compute_validate, _on_complete_success_execution_session

session_batch_status = {}
session_batch_errors = {}
enable_saving_to_disk = False

global_data_root_parent_path = find_first_extant_path([Path(r'W:\Data'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

# Load Pipeline

In [2]:
%pdb off
# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

active_data_mode_name = 'kdiba'

## Data must be pre-processed using the MATLAB script located here: 
#     neuropy/data_session_pre_processing_scripts/KDIBA/IIDataMat_Export_ToPython_2022_08_01.m
# From pre-computed .mat files:

local_session_root_parent_context = IdentifyingContext(format_name=active_data_mode_name) # , animal_name='', configuration_name='one', session_name=self.session_name
local_session_root_parent_path = global_data_root_parent_path.joinpath('KDIBA')

good_contexts_list = [
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54')
]

# %%viztracer
# [*] - indicates bad or session with a problem
# 0, 1, 2, 3, 4, 5, 6, 7, [8], [9], 10, 11, [12], 13, 14, [15], [16], 17, 
# curr_context: IdentifyingContext = good_contexts_list[1] # select the session from all of the good sessions here.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15') # DONE. Very good. Many good Pfs, many good replays.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43') # BAD
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31') # DONE, Good Pfs but no good replays
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6') # BAD
curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19') # DONE, GREAT, both good Pfs and replays!
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3') # DONE, Good Pfs, no good epochs.
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40') # BAD
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3') # BAD, Good Pfs strangely despite horrible map, no good epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59') # BAD, No Epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54') # DONE


local_session_parent_path: Path = local_session_root_parent_path.joinpath(curr_context.animal, curr_context.exper_name) # 'gor01', 'one' - probably not needed anymore
basedir: Path = local_session_parent_path.joinpath(curr_context.session_name).resolve()
print(f'basedir: {str(basedir)}')

# Read if possible:
saving_mode = PipelineSavingScheme.SKIP_SAVING
force_reload = False

# Force write:
# saving_mode = PipelineSavingScheme.TEMP_THEN_OVERWRITE
# saving_mode = PipelineSavingScheme.OVERWRITE_IN_PLACE
# force_reload = True


## TODO: if loading is not possible, we need to change the `saving_mode` so that the new results are properly saved.


    # ==================================================================================================================== #
    # Load Pipeline                                                                                                        #
    # ==================================================================================================================== #
# with VizTracer(output_file=f"viztracer_{get_now_time_str()}-full_session_LOO_decoding_analysis.json", min_duration=200, tracer_entries=3000000, ignore_frozen=True) as tracer:
# epoch_name_includelist = ['maze']
epoch_name_includelist = None
active_computation_functions_name_includelist=[#'_perform_estimated_epochs_computation',  # AL:WAYS OFF
                                            '_perform_baseline_placefield_computation',
                                        # '_perform_time_dependent_placefield_computation', # AL:WAYS OFF
                                        '_perform_extended_statistics_computation',
                                        '_perform_position_decoding_computation', 
                                        '_perform_firing_rate_trends_computation',
                                        '_perform_pf_find_ratemap_peaks_computation',
                                        # '_perform_time_dependent_pf_sequential_surprise_computation' # AL:WAYS OFF
                                        '_perform_two_step_position_decoding_computation',
                                        # '_perform_recursive_latent_placefield_decoding' # AL:WAYS OFF
                                    ]
# active_computation_functions_name_includelist=['_perform_baseline_placefield_computation']

curr_active_pipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, basedir, epoch_name_includelist=epoch_name_includelist,
                                        computation_functions_name_includelist=active_computation_functions_name_includelist,
                                        saving_mode=saving_mode, force_reload=force_reload,
                                        skip_extended_batch_computations=True, debug_print=False, fail_on_exception=True) # , active_pickle_filename = 'loadedSessPickle_withParameters.pkl'


if not force_reload: # not just force_reload, needs to recompute whenever the computation fails.
    try:
        curr_active_pipeline.load_pickled_global_computation_results()
    except Exception as e:
        print(f'cannot load global results: {e}')

# 6m 1.1s
# 12m 15.6s


Automatic pdb calling has been turned OFF
basedir: W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19
Loading loaded session pickle file results : W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\loadedSessPickle.pkl... done.
Loading pickled pipeline success: W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\loadedSessPickle.pkl.
property already present in pickled version. No need to save.
using provided computation_functions_name_includelist: ['_perform_baseline_placefield_computation', '_perform_extended_statistics_computation', '_perform_position_decoding_computation', '_perform_firing_rate_trends_computation', '_perform_pf_find_ratemap_peaks_computation', '_perform_two_step_position_decoding_computation']
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
updating computation_results...
done.
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definit

In [ ]:
%matplotlib qt
from neuropy.utils.matplotlib_helpers import interactive_select_grid_bin_bounds_2D
# fig, ax, rect_selector, set_extents = interactive_select_grid_bin_bounds_2D(curr_active_pipeline, epoch_name='maze', should_block_for_input=True)

grid_bin_bounds = interactive_select_grid_bin_bounds_2D(curr_active_pipeline, epoch_name='maze', should_block_for_input=True, should_apply_updates_to_pipeline=False)
print(f'grid_bin_bounds: {grid_bin_bounds}')
print(f"Add this to `specific_session_override_dict`:\n\n{curr_active_pipeline.get_session_context().get_initialization_code_string()}:dict(grid_bin_bounds=({(grid_bin_bounds[0], grid_bin_bounds[1]), (grid_bin_bounds[2], grid_bin_bounds[3])})),\n")

In [15]:
## Post Compute Validate 2023-05-16:
from scripts.run_BatchAnalysis import post_compute_validate

post_compute_validate(curr_active_pipeline)

were pipeline preprocessing parameters missing and updated?: False


In [16]:
curr_active_pipeline.reload_default_computation_functions()
extended_computations_include_includelist=['long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis', 'long_short_decoding_analyses', 'long_short_post_decoding'] # do only specifiedl , 'long_short_rate_remapping'
force_recompute_global = force_reload
# force_recompute_global = True
# extended_computations_include_includelist=['long_short_post_decoding'] # do only specifiedl
newly_computed_values = batch_extended_computations(curr_active_pipeline, include_includelist=extended_computations_include_includelist, include_global_functions=True, fail_on_exception=True, progress_print=True, force_recompute=force_recompute_global, debug_print=False)
if len(newly_computed_values) > 0:
	print(f'newly_computed_values: {newly_computed_values}. Saving global results...')
	try:
		# curr_active_pipeline.global_computation_results.persist_time = datetime.now()
		# Try to write out the global computation function results:
		curr_active_pipeline.save_global_computation_results()
	except Exception as e:
		print(f'\n\n!!WARNING!!: saving the global results threw the exception: {e}')
		print(f'\tthe global results are currently unsaved! proceed with caution and save as soon as you can!\n\n\n')
else:
	print(f'no changes in global results.')
# 10m 29.5s

included includelist is specified: ['long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis', 'long_short_decoding_analyses', 'long_short_post_decoding'], so only performing these extended computations.
Running batch_extended_computations(...) with global_epoch_name: "maze"
jonathan_firing_rate_analysis already computed.
long_short_fr_indicies_analyses already computed.
long_short_decoding_analyses already computed.
long_short_post_decoding already computed.
done with all batch_extended_computations(...).
no changes in global results.


In [ ]:
curr_active_pipeline.reload_default_display_functions()

neptuner = batch_perform_all_plots(curr_active_pipeline, enable_neptune=True)


### Pipeline Loading/Saving [Optional]

In [ ]:
curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE) # AttributeError: 'PfND_TimeDependent' object has no attribute '_included_thresh_neurons_indx'
# TypeError: cannot pickle 'MplMultiTab' object
# PicklingError: Can't pickle .set_closure_cell at 0x000002BF248F50D0>: it's not found as attr._compat.make_set_closure_cell..set_closure_cell
# TypeError: cannot pickle 'PyQt5.QtCore.pyqtSignal' object

In [ ]:
curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.OVERWRITE_IN_PLACE)

In [ ]:
curr_active_pipeline.save_global_computation_results() # PicklingError: Can't pickle .set_closure_cell at 0x000002BF248F50D0>: it's not found as attr._compat.make_set_closure_cell..set_closure_cell

In [ ]:
curr_active_pipeline.load_pickled_global_computation_results()

In [ ]:
curr_active_pipeline.clear_display_outputs()
curr_active_pipeline.clear_registered_output_files()

In [ ]:
curr_active_pipeline.save_global_computation_results()

# End Run

In [17]:
## long_short_decoding_analyses:
curr_long_short_decoding_analyses = curr_active_pipeline.global_computation_results.computed_data['long_short_leave_one_out_decoding_analysis']
## Extract variables from results object:
long_one_step_decoder_1D, short_one_step_decoder_1D, long_replays, short_replays, global_replays, long_shared_aclus_only_decoder, short_shared_aclus_only_decoder, shared_aclus, long_short_pf_neurons_diff, n_neurons, long_results_obj, short_results_obj, is_global = curr_long_short_decoding_analyses.long_decoder, curr_long_short_decoding_analyses.short_decoder, curr_long_short_decoding_analyses.long_replays, curr_long_short_decoding_analyses.short_replays, curr_long_short_decoding_analyses.global_replays, curr_long_short_decoding_analyses.long_shared_aclus_only_decoder, curr_long_short_decoding_analyses.short_shared_aclus_only_decoder, curr_long_short_decoding_analyses.shared_aclus, curr_long_short_decoding_analyses.long_short_pf_neurons_diff, curr_long_short_decoding_analyses.n_neurons, curr_long_short_decoding_analyses.long_results_obj, curr_long_short_decoding_analyses.short_results_obj, curr_long_short_decoding_analyses.is_global

# (long_one_step_decoder_1D, short_one_step_decoder_1D), (long_one_step_decoder_2D, short_one_step_decoder_2D) = compute_short_long_constrained_decoders(curr_active_pipeline, recalculate_anyway=True)
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
long_epoch_context, short_epoch_context, global_epoch_context = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_epoch_name, short_epoch_name, global_epoch_name)]
long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_computation_config, short_computation_config, global_computation_config = [curr_active_pipeline.computation_results[an_epoch_name]['computation_config'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_pf1D, short_pf1D, global_pf1D = long_results.pf1D, short_results.pf1D, global_results.pf1D
long_pf2D, short_pf2D, global_pf2D = long_results.pf2D, short_results.pf2D, global_results.pf2D
decoding_time_bin_size = long_one_step_decoder_1D.time_bin_size # 1.0/30.0 # 0.03333333333333333

## Get global 'long_short_post_decoding' results:
curr_long_short_post_decoding = curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
expected_v_observed_result, curr_long_short_rr = curr_long_short_post_decoding.expected_v_observed_result, curr_long_short_post_decoding.rate_remapping
rate_remapping_df, high_remapping_cells_only = curr_long_short_rr.rr_df, curr_long_short_rr.high_only_rr_df
Flat_epoch_time_bins_mean, Flat_decoder_time_bin_centers, num_neurons, num_timebins_in_epoch, num_total_flat_timebins, is_short_track_epoch, is_long_track_epoch, short_short_diff, long_long_diff = expected_v_observed_result['Flat_epoch_time_bins_mean'], expected_v_observed_result['Flat_decoder_time_bin_centers'], expected_v_observed_result['num_neurons'], expected_v_observed_result['num_timebins_in_epoch'], expected_v_observed_result['num_total_flat_timebins'], expected_v_observed_result['is_short_track_epoch'], expected_v_observed_result['is_long_track_epoch'], expected_v_observed_result['short_short_diff'], expected_v_observed_result['long_long_diff']

jonathan_firing_rate_analysis_result = JonathanFiringRateAnalysisResult(**curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.to_dict())

# 2023-05-19 - Testing S-only emergence, L-only replays in S, peak position remappings 

In [37]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership, SetPartition
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import TrackExclusivePartitionSubset

# Four distinct subgroups are formed:  pf on neither, pf on both, pf on only long, pf on only short
# L_only_aclus, S_only_aclus

jonathan_firing_rate_analysis_result = JonathanFiringRateAnalysisResult(**curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.to_dict())

neuron_replay_stats_df, short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset = jonathan_firing_rate_analysis_result.get_cell_track_partitions()

In [44]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import _prepare_spikes_df_from_filter_epochs, _find_example_epochs
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership, SetPartition
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import TrackExclusivePartitionSubset
# approach copied from `pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations._epoch_unit_avg_firing_rates`

jonathan_firing_rate_analysis_result = JonathanFiringRateAnalysisResult(**curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.to_dict())
neuron_replay_stats_df, short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset = jonathan_firing_rate_analysis_result.get_cell_track_partitions()

# included_neuron_ids = deepcopy(exclusive_aclus)
# included_neuron_ids = None
included_neuron_ids = EITHER_subset.track_exclusive_aclus
spikes_df: pd.DataFrame = deepcopy(curr_active_pipeline.sess.spikes_df).spikes.sliced_by_neuron_type('pyr')
# filter_epochs = deepcopy(curr_active_pipeline.sess.replay)
# spikes_df = deepcopy(long_results_obj.spikes_df) # LeaveOneOutDecodingAnalysisResult
# spikes_df[np.isin(spikes_df.aclu, included_neuron_ids)]
filter_epochs = deepcopy(long_results_obj.active_filter_epochs)

filter_epoch_spikes_df, filter_epochs_df = _find_example_epochs(spikes_df, filter_epochs, EITHER_subset.track_exclusive_aclus, included_neuron_ids=included_neuron_ids)

In [ ]:
""" 24 cells, 133 Epochs, 789 Total Timebins
# Other
	
	- is_non_firing_time_bin: numpy.ndarray - (24, 789)
	- all_epochs_num_epoch_time_bins: numpy.ndarray - (133,)
	
    ## Indexing Helpers:
		- all_epochs_reverse_flat_epoch_indicies_array: numpy.ndarray - (789,)
		- split_by_epoch_reverse_flattened_time_bin_indicies: list - (133,)
    
	- original_1D_decoder: pyphoplacecellanalysis.Analysis.Decoder.reconstruction.BasePositionDecoder
		- pf: neuropy.analyses.placefields.PfND
		- neuron_IDXs: numpy.ndarray - (24,)
		- neuron_IDs: numpy.ndarray - (24,)
		- F: numpy.ndarray - (119, 24)
		- P_x: numpy.ndarray - (119, 1)
        
	- flat_all_epochs_decoded_epoch_time_bins: numpy.ndarray - (24, 789)

# Measured
	- flat_all_epochs_measured_cell_spike_counts: numpy.ndarray - (24, 789)
	- flat_all_epochs_measured_cell_firing_rates: numpy.ndarray - (24, 789)


# Expected
	- flat_all_epochs_computed_expected_cell_firing_rates: numpy.ndarray - (24, 789)
	- flat_all_epochs_difference_from_expected_cell_spike_counts: numpy.ndarray - (24, 789)
	- flat_all_epochs_difference_from_expected_cell_firing_rates: numpy.ndarray - (24, 789)
    
    
    
## Epoch-based
	- all_epochs_decoded_epoch_time_bins_mean: numpy.ndarray - (133, 24)

    
    
    
- flat_all_epochs_measured_cell_firing_rates: numpy.ndarray - .shape: (24, 789) 
- flat_all_epochs_computed_expected_cell_firing_rates: numpy.ndarray - .shape: (24, 789)


"""

# 2023-04-13 - Find Good looking epochs:

In [8]:
import matplotlib.pyplot as plt
%matplotlib qt
import matplotlib.patheffects as path_effects

from pyphoplacecellanalysis.Pho2D.stacked_epoch_slices import DecodedEpochSlicesPaginatedFigureController

curr_results_obj = long_results_obj
active_identifying_session_ctx = curr_active_pipeline.sess.get_context()
_out_pagination_controller = DecodedEpochSlicesPaginatedFigureController.init_from_decoder_data(curr_results_obj.active_filter_epochs, curr_results_obj.all_included_filter_epochs_decoder_result, 
	xbin=curr_results_obj.original_1D_decoder.xbin, global_pos_df=global_session.position.df, a_name='TestDecodedEpochSlicesPaginationController', active_context=active_identifying_session_ctx,  max_subplots_per_page=200) # 10
# _out_pagination_controller

active_identifying_ctx_string: "kdiba|gor01|two|2006-6-07_16-40-19|1of1|[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132]"


In [ ]:
_out = curr_active_pipeline.display('_display_long_and_short_stacked_epoch_slices')


In [ ]:
## Getting selections out of DecoedEpochSlicesPaginationController
curr_active_pipeline.display_output_history_list
# Ideally could search like: '_display_long_and_short_stacked_epoch_slices'
active_result_backup = curr_active_pipeline.last_added_display_output

In [ ]:
list(active_result_backup.keys()) # ['name', 'context', 'figures', 'axes', 'plot_data']

active_result_backup['context'] # (IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'DecodedEpochSlices', 'replays', 'long_results_obj')>,
								#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'DecodedEpochSlices', 'replays', 'short_results_obj')>)

pagination_controllers = active_result_backup.plot_data['controllers']
long_controller = pagination_controllers[0]
short_controller = pagination_controllers[1]

In [ ]:
long_controller.selected_indicies # np.array([ 13,  14,  15,  25,  27,  28,  31,  37,  42,  45,  48,  57,  61,  62,  63,  76,  79,  82,  89,  90, 111, 112, 113, 115])

# np.array([5,  13,  15,  17,  20,  21,  24,  31,  33,  43,  44,  49,  63, 64,  66,  68,  70,  71,  74,  76,  77,  78,  84,  90,  94,  95, 104, 105, 122, 123])

In [ ]:
short_controller.selected_indicies # np.array([  9,  11,  13,  14,  15,  20,  22,  25,  37,  40,  45,  48,  61, 62,  76,  79,  84,  89,  90,  93,  94, 111, 112, 113, 115, 121])

# np.array([ 12,  13,  15,  17,  20,  24,  30,  31,  32,  33,  41,  43,  49, 54,  55,  68,  70,  71,  73,  76,  77,  78,  84,  89,  94, 100, 104, 105, 111, 114, 115, 117, 118, 122, 123, 131])

In [9]:
final_context_L = curr_active_pipeline.build_display_context_for_session(display_fn_name='DecodedEpochSlices', epochs='replays', decoder='long_results_obj')
final_context_S = curr_active_pipeline.build_display_context_for_session(display_fn_name='DecodedEpochSlices', epochs='replays', decoder='short_results_obj')
# _out_pagination_controller.params.active_identifying_figure_ctx.adding_context(None,  user_annotation="selections")
selections_context_L = final_context_L.adding_context(None,  user_annotation="selections")
selections_context_S = final_context_S.adding_context(None,  user_annotation="selections")

# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19',display_fn_name='DecodedEpochSlices',epochs='replays',decoder='long_results_obj',user_annotation='selections')
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19',display_fn_name='DecodedEpochSlices',epochs='replays',decoder='short_results_obj')

In [ ]:
# Update the `long_results_obj.active_filter_epochs` and `short_results_obj.active_filter_epochs` objects with the selection information
assert long_controller.is_selected.shape[0] == long_results_obj.active_filter_epochs.n_epochs
assert short_controller.is_selected.shape[0] == short_results_obj.active_filter_epochs.n_epochs
# for some reason the active_filter_epochs on both the long and the short objects contain properties about the other one as well, so keep with tradition and add them to both as well
long_results_obj.active_filter_epochs._df['long_is_user_included'] = long_controller.is_selected
long_results_obj.active_filter_epochs._df['short_is_user_included'] = short_controller.is_selected
short_results_obj.active_filter_epochs._df['long_is_user_included'] = long_controller.is_selected
short_results_obj.active_filter_epochs._df['short_is_user_included'] = short_controller.is_selected

In [ ]:
print(list(long_controller.params.keys())) # ['name', 'window_title', 'num_slices', '_debug_test_max_num_slices', 'active_num_slices', 'global_epoch_start_t', 'global_epoch_end_t', 'single_plot_fixed_height', 'all_plots_height', 'epoch_labels', 'variable_name', 'xbin', 'enable_flat_line_drawing', 'debug_print', 'active_identifying_figure_ctx', 'flat_all_data_indicies', 'is_selected', 'callback_id', 'on_render_page_callbacks']

In [ ]:
long_controller.selected_epoch_times
# array([[ 292.62367098,  292.80826736],
#        [ 528.48329294,  528.68561475],
#        [ 572.24842637,  572.44236163],
#        [ 677.99306343,  678.25522777],
#        [ 802.91157867,  803.11436129],
#        [ 831.70962856,  831.90743454],
#        [ 888.22678107,  888.46492239],
#        [ 945.48035303,  945.68543965],
#        [ 954.00517939,  954.2546871 ],
#        [1033.05203044, 1033.22639707],
#        [1061.35727035, 1061.5943979 ],
#        [1214.61082388, 1214.834834  ],
#        [1722.78022002, 1722.95406441],
#        [1741.58530479, 1741.79413924],
#        [1743.25171034, 1743.53445707]])
short_controller.selected_epoch_times
# array([[ 489.42507549,  489.65575186],
#        [ 528.48329294,  528.68561475],
#        [ 572.24842637,  572.44236163],
#        [ 677.99306343,  678.25522777],
#        [ 802.91157867,  803.11436129],
#        [ 888.22678107,  888.46492239],
#        [ 931.21227202,  931.45262518],
#        [ 945.48035303,  945.68543965],
#        [ 950.18327543,  950.44046313],
#        [ 954.00517939,  954.2546871 ],
#        [1024.83173129, 1025.10320379],
#        [1033.05203044, 1033.22639707],
#        [1214.61082388, 1214.834834  ],
#        [1326.54476977, 1326.76506278],
#        [1329.84606216, 1330.0684134 ],
#        [1732.09310613, 1732.29060491],
#        [1741.58530479, 1741.79413924],
#        [1743.25171034, 1743.53445707],
#        [1751.03108555, 1751.22059714],
#        [1769.90424581, 1770.10497019],
#        [1776.56793264, 1776.78871716],
#        [1871.00426701, 1871.21629634],
#        [1917.09385877, 1917.2703758 ],
#        [1994.07242543, 1994.2525367 ],
#        [2038.53127755, 2038.70899266],
#        [2064.36693792, 2064.54016792],
#        [2111.52742834, 2111.80120484],
#        [2125.54830564, 2125.73772506]])

In [ ]:
long_results_obj.active_filter_epochs.n_epochs # 122

# TODO 2023-06-19 14:31: - [ ] Come up with system for storing user-annotations:
 Need to store user-annotations and labeled selections for Epochs:

	date_annotated: datetime - the date the annotations were made
	annotation_author: str - the person who made the annotations
	annotation_content: Any - the content of the annotation



In [9]:
# All epochs are shown for both decoders. So the result should be an epochs table with separate columns for the inclusion in each decoder.
@define(slots=False, eq=False)
class EpochSelectionSet:
    selected_indicies: np.ndarray # a list of the selected inidices

long_selection = EpochSelectionSet(selected_indicies=np.array([5,  13,  15,  17,  20,  21,  24,  31,  33,  43,  44,  49,  63, 64,  66,  68,  70,  71,  74,  76,  77,  78,  84,  90,  94,  95, 104, 105, 122, 123]))
short_selection = EpochSelectionSet(selected_indicies=np.array([ 12,  13,  15,  17,  20,  24,  30,  31,  32,  33,  41,  43,  49, 54,  55,  68,  70,  71,  73,  76,  77,  78,  84,  89,  94, 100, 104, 105, 111, 114, 115, 117, 118, 122, 123, 131]))

In [12]:
# For updating the controller from the selected indicies:
for a_selected_index in selection_idxs_L:
	_out_pagination_controller.params.is_selected[a_selected_index] = True

_out_pagination_controller.perform_update_selections(defer_render=False)

In [50]:
from pyphoplacecellanalysis.GUI.Qt.Mixins.PaginationMixins import SelectionsObject
import cattrs

user_annotations = {}

## IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15')
user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15',display_fn_name='DecodedEpochSlices',epochs='replays',decoder='long_results_obj',user_annotation='selections')] = np.array([ 13,  14,  15,  25,  27,  28,  31,  37,  42,  45,  48,  57,  61,  62,  63,  76,  79,  82,  89,  90, 111, 112, 113, 115])
user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15',display_fn_name='DecodedEpochSlices',epochs='replays',decoder='short_results_obj',user_annotation='selections')] = np.array([  9,  11,  13,  14,  15,  20,  22,  25,  37,  40,  45,  48,  61, 62,  76,  79,  84,  89,  90,  93,  94, 111, 112, 113, 115, 121])

## IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19')
user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19',display_fn_name='DecodedEpochSlices',epochs='replays',decoder='long_results_obj',user_annotation='selections')] = np.array([5,  13,  15,  17,  20,  21,  24,  31,  33,  43,  44,  49,  63, 64,  66,  68,  70,  71,  74,  76,  77,  78,  84,  90,  94,  95, 104, 105, 122, 123])
user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19',display_fn_name='DecodedEpochSlices',epochs='replays',decoder='short_results_obj',user_annotation='selections')] = np.array([ 12,  13,  15,  17,  20,  24,  30,  31,  32,  33,  41,  43,  49, 54,  55,  68,  70,  71,  73,  76,  77,  78,  84,  89,  94, 100, 104, 105, 111, 114, 115, 117, 118, 122, 123, 131])


## try to get the user annotations for this session:
selection_idxs_L = user_annotations[selections_context_L]
selection_idxs_S = user_annotations[selections_context_S]

In [53]:
# cattrs.unstructure(user_annotations)
# cattrs.unstructure(selection_idxs_S)
cattrs.unstructure(selections_context_L)

{}

In [55]:
selections_context_L.as_tuple()

('kdiba',
 'gor01',
 'two',
 '2006-6-07_16-40-19',
 'DecodedEpochSlices',
 'replays',
 'long_results_obj',
 'selections')

In [46]:
# for updating the filter_epochs_df (`filter_epochs_df`) from the selections:
filter_epochs_df['long_is_user_included'] = np.isin(filter_epochs_df.index, selection_idxs_L)
filter_epochs_df['short_is_user_included'] = np.isin(filter_epochs_df.index, selection_idxs_S)

## Finally filter to find the potential example epochs:

# only include those with one or more exclusive aclu:
considered_filter_epochs_df = filter_epochs_df[filter_epochs_df.contains_one_exclusive_aclu].copy()
# require inclusion for long or short:
considered_filter_epochs_df = considered_filter_epochs_df[np.logical_xor(filter_epochs_df['long_is_user_included'], filter_epochs_df['short_is_user_included'])]
considered_filter_epochs_df


,start,stop,label,duration,score,velocity,intercept,speed,score_LONG,velocity_LONG,intercept_LONG,speed_LONG,score_SHORT,velocity_SHORT,intercept_SHORT,speed_SHORT,contains_one_exclusive_aclu,long_is_user_included,short_is_user_included
5,292.623671,292.808267,5,0.184596,0.432036,341.969272,24.463794,341.969272,0.432036,341.969272,24.463794,341.969272,0.529333,190.906675,59.653723,190.906675,True,True,False
12,489.425075,489.655752,12,0.230676,0.610818,-344.372039,188.867342,344.372039,0.610818,-344.372039,188.867342,344.372039,0.881344,-299.849982,174.402042,299.849982,True,False,True
21,831.709629,831.907435,21,0.197806,0.729495,-458.220009,192.398514,458.220009,0.729495,-458.220009,192.398514,458.220009,0.874295,-203.417377,157.729123,203.417377,True,True,False
30,931.212272,931.452625,30,0.240353,0.595731,-520.883403,220.829261,520.883403,0.595731,-520.883403,220.829261,520.883403,0.676472,-358.879084,184.170410,358.879084,True,False,True
32,950.183275,950.440463,32,0.257188,0.412421,-748.222887,215.339304,748.222887,0.412421,-748.222887,215.339304,748.222887,0.710759,-767.364559,193.652110,767.364559,True,False,True
41,1024.831731,1025.103204,41,0.271473,0.575993,-275.028224,97.919029,275.028224,0.575993,-275.028224,97.919029,275.028224,0.798317,-283.546958,135.898789,283.546958,True,False,True
44,1061.357270,1061.594398,44,0.237128,0.481308,-1048.138239,238.875671,1048.138239,0.481308,-1048.138239,238.875671,1048.138239,0.646727,-634.178688,188.606751,634.178688,True,True,False
54,1326.544770,1326.765063,54,0.220293,0.489237,318.737270,21.218745,318.737270,0.489237,318.737270,21.218745,318.737270,0.758235,630.523133,43.235530,630.523133,True,False,True
55,1329.846062,1330.068413,55,0.222351,0.488757,994.323723,34.049264,994.323723,0.488757,994.323723,34.049264,994.323723,0.491774,521.311916,76.140611,521.311916,True,False,True
63,1562.800403,1562.970930,63,0.170527,0.530699,404.462603,89.605391,404.462603,0.530699,404.462603,89.605391,404.462603,0.508334,515.834286,63.857079,515.834286,True,True,False


# Other Programmatic Figures

In [ ]:
curr_active_pipeline.reload_default_display_functions()

In [ ]:
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons

## Builds proper sort indicies for '_display_short_long_pf1D_comparison'
long_ratemap = long_pf1D.ratemap
short_ratemap = short_pf1D.ratemap

curr_any_context_neurons = _find_any_context_neurons(*[k.neuron_ids for k in [long_ratemap, short_ratemap]])
curr_any_context_neurons

In [ ]:
from neuropy.plotting.ratemaps import plot_ratemap_1D
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_short_v_long_pf1D_comparison, LongShortTrackComparingDisplayFunctions

curr_active_pipeline.reload_default_display_functions()

%matplotlib qt
active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'

long_single_cell_pfmap_processing_fn = None
short_single_cell_pfmap_processing_fn = None
# sort_idx = None
# sort_idx = sort_ind
# sort_idx = sort_indicies
# sort_idx = new_all_aclus_sort_indicies
sort_idx = 'peak_long'
# sort_idx = 'bad'

# out = curr_active_pipeline.display('_display_short_long_pf1D_comparison', active_identifying_session_ctx, single_figure=False, debug_print=True, fignum='Short v Long pf1D Comparison',
#                                    long_kwargs={'included_unit_neuron_IDs': curr_any_context_neurons, 'included_unit_indicies': None, 'sortby': sort_idx, 'single_cell_pfmap_processing_fn': long_single_cell_pfmap_processing_fn},
#                                    short_kwargs={'included_unit_neuron_IDs': curr_any_context_neurons, 'included_unit_indicies': None,'sortby': sort_idx, 'single_cell_pfmap_processing_fn': short_single_cell_pfmap_processing_fn, 'curve_hatch_style': {'hatch':'///', 'edgecolor':'k'}},
#                                   )

out = curr_active_pipeline.display('_display_short_long_pf1D_comparison', active_identifying_session_ctx, single_figure=False, debug_print=False, fignum='Short v Long pf1D Comparison',
                                   long_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': long_single_cell_pfmap_processing_fn},
                                   short_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': short_single_cell_pfmap_processing_fn, 'curve_hatch_style': {'hatch':'///', 'edgecolor':'k'}},
                                  )
                                  

# ax = out.axes[0]

In [ ]:
curr_active_pipeline.display('_display_short_long_pf1D_comparison', single_figure=False)

# 2023-06-01 - Testing new metrics:

In [ ]:

curr_long_short_post_decoding = curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
## Extract variables from results object:
expected_v_observed_result, curr_long_short_rr = curr_long_short_post_decoding.expected_v_observed_result, curr_long_short_post_decoding.rate_remapping
rate_remapping_df, high_remapping_cells_only = curr_long_short_rr.rr_df, curr_long_short_rr.high_only_rr_df
Flat_epoch_time_bins_mean, Flat_decoder_time_bin_centers, num_neurons, num_timebins_in_epoch, num_total_flat_timebins, is_short_track_epoch, is_long_track_epoch, short_short_diff, long_long_diff = expected_v_observed_result['Flat_epoch_time_bins_mean'], expected_v_observed_result['Flat_decoder_time_bin_centers'], expected_v_observed_result['num_neurons'], expected_v_observed_result['num_timebins_in_epoch'], expected_v_observed_result['num_total_flat_timebins'], expected_v_observed_result['is_short_track_epoch'], expected_v_observed_result['is_long_track_epoch'], expected_v_observed_result['short_short_diff'], expected_v_observed_result['long_long_diff']

print(f'long_test: {np.mean(long_long_diff)}')
print(f'short_test: {np.mean(short_short_diff)}')

In [ ]:
# long_results_obj.original_1D_decoder
replay_result_df = deepcopy(long_results_obj.active_filter_epochs.to_dataframe())
replay_result_df

In [ ]:
short_results_obj.active_filter_epochs.to_dataframe()

In [ ]:
# Look at lap speed over time - doesn't look like there's a difference between the long and the short track speeds
global_session.position.compute_higher_order_derivatives()
running_pos_df = global_session.position.to_dataframe()
ax = running_pos_df.plot(x='t', y=['lin_pos','speed'], title='Running Speed over Time')
ax

In [ ]:
# Call the function
fig, ax = plot_lines(replay_result_df)

# 2023-06-13 11:01: - [ ] New display function testing

In [47]:
import matplotlib as mpl
# from pyphoplacecellanalysis.General.Mixins.ExportHelpers import create_daily_programmatic_display_function_testing_folder_if_needed, session_context_to_relative_path, programmatic_display_to_PDF
# from pyphoplacecellanalysis.General.Mixins.ExportHelpers import build_pdf_metadata_from_display_context # newer version of build_pdf_export_metadata
from neuropy.core.neuron_identities import PlotStringBrevityModeEnum
from neuropy.plotting.ratemaps import BackgroundRenderingOptions
_bak_rcParams = mpl.rcParams.copy()
# mpl.rcParams['toolbar'] = 'None' # disable toolbars
# matplotlib.use('AGG') # non-interactive backend ## 2022-08-16 - Surprisingly this works to make the matplotlib figures render only to .png file, not appear on the screen!

%matplotlib qt

from pyphoplacecellanalysis.General.Mixins.ExportHelpers import programmatic_render_to_file

In [48]:
curr_active_pipeline.reload_default_display_functions()


In [55]:
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap, visualize_heatmap_pyqtgraph # used in `plot_kourosh_activity_style_figure`
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_raster_plot # used in `plot_kourosh_activity_style_figure`
import pyphoplacecellanalysis.External.pyqtgraph as pg # used in `plot_kourosh_activity_style_figure`

heatmap_pf1D_win, heatmap_pf1D_img = visualize_heatmap_pyqtgraph(long_pf1D.ratemap.normalized_tuning_curves, title=f"pf1D Visualization")
# Apply the colormap
# heatmap_pf1D_img.setLookupTable(lut)
heatmap_pf1D_win.show()

In [ ]:
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import _prepare_spikes_df_from_filter_epochs

long_ratemap = long_pf1D.ratemap
short_ratemap = short_pf1D.ratemap
curr_any_context_neurons = _find_any_context_neurons(*[k.neuron_ids for k in [long_ratemap, short_ratemap]])

# filter_epochs = deepcopy(long_replays)

filter_epochs = deepcopy(long_results_obj.active_filter_epochs.to_dataframe())
filter_epoch_spikes_df = _prepare_spikes_df_from_filter_epochs(long_session.spikes_df, filter_epochs=filter_epochs, included_neuron_ids=curr_any_context_neurons, epoch_id_key_name='replay_epoch_id', debug_print=False) # replay_epoch_id
filter_epoch_spikes_df

In [ ]:
app, win, plots, plots_data = plot_multiple_raster_plot(filter_epochs_df, filter_epoch_spikes_df, included_neuron_ids=shared_aclus, epoch_id_key_name='replay_epoch_id', scatter_app_name="Pho Stacked Replays")

In [94]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_multiple_raster_plot
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import determine_long_short_pf1D_indicies_sort_by_peak
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import _prepare_spikes_df_from_filter_epochs, _find_example_epochs
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership, SetPartition
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import TrackExclusivePartitionSubset
# approach copied from `pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations._epoch_unit_avg_firing_rates`

jonathan_firing_rate_analysis_result = JonathanFiringRateAnalysisResult(**curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.to_dict())
neuron_replay_stats_df, short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset = jonathan_firing_rate_analysis_result.get_cell_track_partitions()

# included_neuron_ids = deepcopy(exclusive_aclus)
# included_neuron_ids = None
included_neuron_ids = EITHER_subset.track_exclusive_aclus
spikes_df: pd.DataFrame = deepcopy(curr_active_pipeline.sess.spikes_df).spikes.sliced_by_neuron_type('pyr')
# filter_epochs = deepcopy(curr_active_pipeline.sess.replay)
# spikes_df = deepcopy(long_results_obj.spikes_df) # LeaveOneOutDecodingAnalysisResult
# spikes_df[np.isin(spikes_df.aclu, included_neuron_ids)]
filter_epochs_df = deepcopy(long_results_obj.active_filter_epochs.to_dataframe())


In [96]:

filter_epoch_spikes_df, filter_epochs_df = _find_example_epochs(spikes_df, filter_epochs_df, EITHER_subset.track_exclusive_aclus, included_neuron_ids=included_neuron_ids) # adds 'active_unique_aclus'
# epoch_contains_any_exclusive_aclus.append(np.isin(epoch_spikes_unique_aclus, exclusive_aclus).any())
filter_epochs_df['has_SHORT_exclusive_aclu'] = [np.isin(epoch_spikes_unique_aclus, short_exclusive.track_exclusive_aclus).any() for epoch_spikes_unique_aclus in filter_epochs_df['active_unique_aclus']]
filter_epochs_df['has_LONG_exclusive_aclu'] = [np.isin(epoch_spikes_unique_aclus, long_exclusive.track_exclusive_aclus).any() for epoch_spikes_unique_aclus in filter_epochs_df['active_unique_aclus']]

# requires selection_idxs_L, selection_idxs_S

# for updating the filter_epochs_df (`filter_epochs_df`) from the selections:
filter_epochs_df['long_is_user_included'] = np.isin(filter_epochs_df.index, selection_idxs_L)
filter_epochs_df['short_is_user_included'] = np.isin(filter_epochs_df.index, selection_idxs_S)

# Finally, get only the epochs that meet the criteria:


# # only include those with one or more exclusive aclu:
# considered_filter_epochs_df = filter_epochs_df[filter_epochs_df.contains_one_exclusive_aclu].copy()
# # require inclusion for long or short:
# considered_filter_epochs_df = considered_filter_epochs_df[np.logical_xor(filter_epochs_df['long_is_user_included'], filter_epochs_df['short_is_user_included'])]

# Get separate long-side/short-side canidate replays:
# considered_long_side_epochs_df = filter_epochs_df[(filter_epochs_df['has_LONG_exclusive_aclu'] & ~filter_epochs_df['short_is_user_included'] & filter_epochs_df['long_is_user_included'])].copy() # replay not considered good by user for short decoding, but it is for long decoding. Finally, has at least one LONG exclusive ACLU.
considered_long_side_epochs_df = filter_epochs_df[(filter_epochs_df['has_LONG_exclusive_aclu'] & filter_epochs_df['long_is_user_included'])].copy() # replay not considered good by user for short decoding, but it is for long decoding. Finally, has at least one LONG exclusive ACLU.
# considered_long_side_epochs_df
# considered_short_side_epochs_df = filter_epochs_df[(filter_epochs_df['has_SHORT_exclusive_aclu'] & filter_epochs_df['short_is_user_included'] & ~filter_epochs_df['long_is_user_included'])].copy()  # replay not considered good by user for long decoding, but it is for short decoding. Finally, has at least one SHORT exclusive ACLU.
considered_short_side_epochs_df = filter_epochs_df[(filter_epochs_df['has_SHORT_exclusive_aclu'] & filter_epochs_df['short_is_user_included'])].copy()  # replay not considered good by user for long decoding, but it is for short decoding. Finally, has at least one SHORT exclusive ACLU.
# considered_short_side_epochs_df


# Common for all rasters:
new_all_aclus_sort_indicies = determine_long_short_pf1D_indicies_sort_by_peak(curr_active_pipeline=curr_active_pipeline, curr_any_context_neurons=EITHER_subset.track_exclusive_aclus)

# considered_filter_epochs_df = deepcopy(considered_long_side_epochs_df)
considered_filter_epochs_df = deepcopy(considered_short_side_epochs_df)
# considered_filter_epochs_df = deepcopy(filter_epochs_df)


filter_epochs: 2
np.shape(filter_epoch_spikes_df): (4949, 24)
np.shape(filter_epoch_spikes_df): (98, 24)


In [ ]:

filter_epoch_spikes_df = _prepare_spikes_df_from_filter_epochs(filter_epoch_spikes_df, filter_epochs=considered_filter_epochs_df, included_neuron_ids=EITHER_subset.track_exclusive_aclus, epoch_id_key_name='replay_epoch_id', debug_print=True) # replay_epoch_id
app, win, plots, plots_data = plot_multiple_raster_plot(considered_filter_epochs_df, filter_epoch_spikes_df, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, epoch_id_key_name='replay_epoch_id', scatter_app_name="Pho Custom Selected Stacked Replays")

In [97]:
active_filter_epochs = deepcopy(considered_short_side_epochs_df)
filter_epoch_spikes_df = _prepare_spikes_df_from_filter_epochs(filter_epoch_spikes_df, filter_epochs=active_filter_epochs, included_neuron_ids=EITHER_subset.track_exclusive_aclus, epoch_id_key_name='replay_epoch_id', debug_print=True) # replay_epoch_id
app, win, plots, plots_data = plot_multiple_raster_plot(active_filter_epochs, filter_epoch_spikes_df, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, epoch_id_key_name='replay_epoch_id', scatter_app_name="Pho Custom Selected Stacked Replays - Short Side")

filter_epochs: 21
np.shape(filter_epoch_spikes_df): (98, 26)
np.shape(filter_epoch_spikes_df): (66, 26)


In [98]:
active_filter_epochs = deepcopy(considered_long_side_epochs_df)
filter_epoch_spikes_df = _prepare_spikes_df_from_filter_epochs(filter_epoch_spikes_df, filter_epochs=active_filter_epochs, included_neuron_ids=EITHER_subset.track_exclusive_aclus, epoch_id_key_name='replay_epoch_id', debug_print=True) # replay_epoch_id
app, win, plots, plots_data = plot_multiple_raster_plot(active_filter_epochs, filter_epoch_spikes_df, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, epoch_id_key_name='replay_epoch_id', scatter_app_name="Pho Custom Selected Stacked Replays - LONG Side")

filter_epochs: 2
np.shape(filter_epoch_spikes_df): (66, 26)
np.shape(filter_epoch_spikes_df): (66, 26)


In [60]:
filter_epochs_df

,start,stop,label,duration,score,velocity,intercept,speed,score_LONG,velocity_LONG,intercept_LONG,speed_LONG,score_SHORT,velocity_SHORT,intercept_SHORT,speed_SHORT,contains_one_exclusive_aclu,active_unique_aclus,long_is_user_included,short_is_user_included
0,49.107308,49.285484,0,0.178176,0.473504,139.775115,14.282968,139.775115,0.473504,139.775115,14.282968,139.775115,0.394694,-75.746823,144.371176,75.746823,True,"[7, 27, 20, 45, 34, 52, 60, 21, 63, 8]",False,False
1,68.571951,68.764442,1,0.192491,0.481259,-76.113799,215.237778,76.113799,0.481259,-76.113799,215.237778,76.113799,0.389931,-405.081114,233.286214,405.081114,True,"[64, 7, 56, 34, 6, 60, 21, 63]",False,False
2,91.226833,91.413549,2,0.186716,0.479736,-176.234177,215.701002,176.234177,0.479736,-176.234177,215.701002,176.234177,0.469504,-174.763795,170.367746,174.763795,True,"[34, 64, 8, 20, 35, 28, 39, 51, 6, 53, 60, 17,...",False,False
3,235.576353,235.756188,3,0.179835,0.514614,-309.395869,168.113304,309.395869,0.514614,-309.395869,168.113304,309.395869,0.640654,-225.084695,158.472326,225.084695,True,"[56, 53, 11, 7, 17, 45, 6, 62, 20, 52, 65, 39,...",False,False
4,272.462145,272.641058,4,0.178913,0.487484,143.145623,195.675563,143.145623,0.487484,143.145623,195.675563,143.145623,0.458075,276.852686,157.437154,276.852686,True,"[35, 64, 34, 21, 8, 6, 60, 52, 20]",False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,2503.348263,2503.557405,128,0.209142,0.498792,275.305437,167.463235,275.305437,0.498792,275.305437,167.463235,275.305437,0.506303,131.875685,154.790445,131.875685,True,"[60, 34, 27, 39, 20, 25, 35, 8, 6, 31, 40, 49,...",False,False
129,2520.181371,2520.376197,129,0.194826,0.743931,-210.206581,204.239042,210.206581,0.743931,-210.206581,204.239042,210.206581,0.601656,16.137238,164.430661,16.137238,True,"[25, 34, 8, 21, 39, 35, 60, 6, 53, 17, 7, 45, ...",False,False
130,2525.535096,2525.740674,130,0.205578,0.334451,14.879566,31.495167,14.879566,0.334451,14.879566,31.495167,14.879566,0.519752,453.761591,50.101389,453.761591,True,"[28, 35, 60, 53, 63, 8, 49, 6, 7, 40, 62, 20, ...",False,False
131,2530.724715,2530.918773,131,0.194058,0.706966,255.289901,20.031194,255.289901,0.706966,255.289901,20.031194,255.289901,0.983988,162.225189,54.724223,162.225189,True,"[7, 52, 26, 39, 28, 41, 63, 19, 8, 60, 40, 64,...",False,True


In [85]:
considered_short_side_epochs_df

,start,stop,label,duration,score,velocity,intercept,speed,score_LONG,velocity_LONG,...,score_SHORT,velocity_SHORT,intercept_SHORT,speed_SHORT,contains_one_exclusive_aclu,active_unique_aclus,has_SHORT_exclusive_aclu,has_LONG_exclusive_aclu,long_is_user_included,short_is_user_included
55,1329.846062,1330.068413,55,0.222351,0.488757,994.323723,34.049264,994.323723,0.488757,994.323723,...,0.491774,521.311916,76.140611,521.311916,True,"[7, 26, 51, 39, 20, 62, 63, 40, 60, 55, 18, 11...",True,True,False,True
73,1751.031086,1751.220597,73,0.189512,0.561078,-1130.449851,210.757919,1130.449851,0.561078,-1130.449851,...,0.603451,-728.733840,181.755222,728.733840,True,"[20, 34, 25, 21, 39, 56, 17, 35, 11, 8, 43, 31...",True,True,False,True
89,1994.072425,1994.252537,89,0.180111,0.519541,-160.527574,200.449171,160.527574,0.519541,-160.527574,...,0.696647,-465.508549,189.575491,465.508549,True,"[20, 45, 64, 34, 39, 7, 35, 44, 6, 53, 17, 11,...",True,False,False,True
100,2064.366938,2064.540168,100,0.173230,0.489089,-90.438050,196.769997,90.438050,0.489089,-90.438050,...,0.563679,-636.603053,183.010550,636.603053,True,"[55, 34, 8, 35, 39, 17, 11, 7, 49, 40, 31, 51,...",True,False,False,True
111,2192.122751,2192.370385,111,0.247634,0.400525,238.678884,151.602404,238.678884,0.400525,238.678884,...,0.487787,873.311468,22.045000,873.311468,True,"[52, 28, 39, 7, 26, 63, 19, 40, 49, 64, 11, 34...",True,False,False,True
114,2198.072536,2198.259713,114,0.187177,0.474458,-396.463340,94.231566,396.463340,0.474458,-396.463340,...,0.711757,-390.618569,111.254595,390.618569,True,"[49, 40, 35, 64, 19, 41, 52, 65, 39, 51, 26, 7...",True,False,False,True
117,2214.245655,2214.441127,117,0.195471,0.333104,-80.662979,112.689308,80.662979,0.333104,-80.662979,...,0.622055,-754.136279,163.942122,754.136279,True,"[17, 34, 53, 7, 56, 35, 27, 40, 18, 19, 41, 60...",True,False,False,True
118,2248.612655,2248.809693,118,0.197038,0.542158,-177.538102,196.553081,177.538102,0.542158,-177.538102,...,0.673981,-491.013925,178.030036,491.013925,True,"[34, 25, 8, 24, 35, 17, 53, 6, 39, 11, 31, 49,...",True,False,False,True
131,2530.724715,2530.918773,131,0.194058,0.706966,255.289901,20.031194,255.289901,0.706966,255.289901,...,0.983988,162.225189,54.724223,162.225189,True,"[7, 52, 26, 39, 28, 41, 63, 19, 8, 60, 40, 64,...",True,False,False,True


In [86]:
considered_long_side_epochs_df

,start,stop,label,duration,score,velocity,intercept,speed,score_LONG,velocity_LONG,...,score_SHORT,velocity_SHORT,intercept_SHORT,speed_SHORT,contains_one_exclusive_aclu,active_unique_aclus,has_SHORT_exclusive_aclu,has_LONG_exclusive_aclu,long_is_user_included,short_is_user_included


In [70]:
considered_short_side_epochs_df = filter_epochs_df[ (filter_epochs_df['has_SHORT_exclusive_aclu'] & filter_epochs_df['short_is_user_included'] & ~filter_epochs_df['long_is_user_included']) ]  # replay not considered good by user for long decoding, but it is for short decoding. Finally, has at least one SHORT exclusive ACLU.
considered_short_side_epochs_df

considered_long_side_epochs_df = filter_epochs_df[ (filter_epochs_df['has_LONG_exclusive_aclu'] & ~filter_epochs_df['short_is_user_included'] & filter_epochs_df['long_is_user_included']) ] # replay not considered good by user for short decoding, but it is for long decoding. Finally, has at least one LONG exclusive ACLU.
considered_long_side_epochs_df

,start,stop,label,duration,score,velocity,intercept,speed,score_LONG,velocity_LONG,...,score_SHORT,velocity_SHORT,intercept_SHORT,speed_SHORT,contains_one_exclusive_aclu,active_unique_aclus,long_is_user_included,short_is_user_included,has_SHORT_exclusive_aclu,has_LONG_exclusive_aclu
0,49.107308,49.285484,0,0.178176,0.473504,139.775115,14.282968,139.775115,0.473504,139.775115,...,0.394694,-75.746823,144.371176,75.746823,True,"[7, 27, 20, 45, 34, 52, 60, 21, 63, 8]",True,False,False,True
1,68.571951,68.764442,1,0.192491,0.481259,-76.113799,215.237778,76.113799,0.481259,-76.113799,...,0.389931,-405.081114,233.286214,405.081114,True,"[64, 7, 56, 34, 6, 60, 21, 63]",True,False,False,True
4,272.462145,272.641058,4,0.178913,0.487484,143.145623,195.675563,143.145623,0.487484,143.145623,...,0.458075,276.852686,157.437154,276.852686,True,"[35, 64, 34, 21, 8, 6, 60, 52, 20]",True,False,False,True
6,314.248451,314.448653,6,0.200202,0.655868,89.701526,195.685022,89.701526,0.655868,89.701526,...,0.576544,-45.044663,177.548494,45.044663,True,"[64, 35, 34, 8, 21, 15, 6, 60, 65, 39, 20]",True,False,True,True
8,470.135813,470.317983,8,0.182170,0.555534,-196.556797,200.308355,196.556797,0.555534,-196.556797,...,0.663955,-340.284911,178.755688,340.284911,True,"[34, 25, 21, 6, 11, 7, 60, 45, 20, 31, 40, 41]",True,False,False,True
9,476.709183,476.899555,9,0.190372,0.606170,96.212154,186.087513,96.212154,0.606170,96.212154,...,0.616901,52.895729,164.823072,52.895729,True,"[34, 53, 55, 60, 7, 51, 8, 21, 25, 35, 6, 39, ...",True,False,False,True
14,558.985903,559.257744,14,0.271841,0.477524,130.412017,183.165829,130.412017,0.477524,130.412017,...,0.517671,152.128503,143.343039,152.128503,True,"[60, 34, 17, 53, 6, 7, 11, 45, 35, 25, 21, 64,...",True,False,False,True
16,672.955600,673.222250,16,0.266649,0.839023,-136.114360,216.388038,136.114360,0.839023,-136.114360,...,0.754317,-92.303095,179.167372,92.303095,True,"[25, 34, 8, 21, 35, 15, 39, 64, 60, 17, 53, 7,...",True,False,False,True
18,705.771916,706.021578,18,0.249661,0.825458,-69.745099,201.908191,69.745099,0.825458,-69.745099,...,0.861188,-93.078822,182.964289,93.078822,True,"[64, 8, 35, 34, 27, 25, 39, 60, 21, 53, 17, 7,...",True,False,False,True
25,892.692668,892.913238,25,0.220569,0.553147,78.225078,197.533853,78.225078,0.553147,78.225078,...,0.546953,77.695962,159.497874,77.695962,True,"[8, 34, 56, 35, 21, 25, 39, 6, 20, 7]",True,False,False,True


In [ ]:
def PAPER_FIGURE_figure_1_add_replay_epoch_rasters(considered_filter_epochs_df):
